In [123]:
from openai import OpenAI
from dotenv import dotenv_values
import json
# **** GPT_MODEL **** #
GPT_MODEL = "gpt-3.5-turbo-1106"


OPENAI_API_KEY:str = dotenv_values(".env").get("OPENAI_API_KEY")
client:OpenAI = OpenAI(api_key=OPENAI_API_KEY)


In [124]:
def getCurrentWeather(location:str, unit:str="fahrenheit")->str | dict | None:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location

In [125]:
def show_json(obj):
    return json.loads(obj.model_dump_json())

In [126]:
from openai.types.beta.assistant import Assistant
from typing import Union

tools:list[dict[str, Union[str, dict]]] = [
    {
        "type":"function",
        "function":{
            "name":"getCurrentWeather",
            "description":"Get the weather in location",
            "parameters":{
                "type":"object",
                "properties":{
                   "location":{"type":"string", "description":"The city and state e.g. San Francisco, CA"},
                   "unit":{"type":"string", "enum":["f","c"]}
                },
                "required":["location"]
            }
        }
    },
     {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  }
]

assistant:Assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model=GPT_MODEL,
    tools=tools
)

show_json(assistant)

{'id': 'asst_hC4omOMeVgs4Lk8ZoMWO8Giz',
 'created_at': 1702644924,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [{'function': {'name': 'getCurrentWeather',
    'parameters': {'type': 'object',
     'properties': {'location': {'type': 'string',
       'description': 'The city and state e.g. San Francisco, CA'},
      'unit': {'type': 'string', 'enum': ['f', 'c']}},
     'required': ['location']},
    'description': 'Get the weather in location'},
   'type': 'function'},
  {'function': {'name': 'getNickname',
    'parameters': {'type': 'object',
     'properties': {'location': {'type': 'string',
       'description': 'The city and state e.g. San Francisco, CA'}},
     'required': ['location']},
    'description': 'Get the nickname of a city'},
   'type': 'function'}]}

In [127]:
from openai.types.beta.thread import Thread

thread:Thread =  client.beta.threads.create()

show_json(thread)

{'id': 'thread_KX8YikJXTQ1W5inLfj3D7ohu',
 'created_at': 1702644924,
 'metadata': {},
 'object': 'thread'}

In [128]:
from openai.types.beta.threads.thread_message import ThreadMessage

message:ThreadMessage =  client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="how is the weather in Los Angles?"
)

show_json(message)

{'id': 'msg_kCczYVE2vVt4M3Jaj7zCQr3n',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'how is the weather in Los Angles?'},
   'type': 'text'}],
 'created_at': 1702644925,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_KX8YikJXTQ1W5inLfj3D7ohu'}

In [129]:
from openai.types.beta.threads.run import Run

run:Run =  client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [130]:
show_json(run)

{'id': 'run_pggKfBt0qxu2XywYQyCL0AEF',
 'assistant_id': 'asst_hC4omOMeVgs4Lk8ZoMWO8Giz',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702644926,
 'expires_at': 1702645526,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_KX8YikJXTQ1W5inLfj3D7ohu',
 'tools': [{'function': {'name': 'getCurrentWeather',
    'parameters': {'type': 'object',
     'properties': {'location': {'type': 'string',
       'description': 'The city and state e.g. San Francisco, CA'},
      'unit': {'type': 'string', 'enum': ['f', 'c']}},
     'required': ['location']},
    'description': 'Get the weather in location'},
   'type': 'function'},
  {'function': {'name': 'getNickname',
    'parameters': {'type': 'object',
     'properties': {'lo

In [131]:
available_functions = {
    "getCurrentWeather":getCurrentWeather,
    "getNickname":getNickname
}

In [132]:
import time

while True:
    runRequest = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
        # if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
# 
    # run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)

    if runRequest.status == "requires_action":
        print(runRequest.status,"==========")
        print("Status: ", "requires_action")
        display(show_json(runRequest.required_action))
        if runRequest.required_action.submit_tool_outputs and runRequest.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls Present:")
            toolCalls = runRequest.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            print(toolCalls)
            for toolcall in toolCalls:
                # print("-----------")
                # display(show_json(toolcall))
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)

                if function_name in available_functions:
                    function_to_call =  available_functions[function_name]

                    if(function_to_call.__name__ == "getCurrentWeather"):
                        print(function_name)
                        response =  function_to_call(
                            location=function_args.get("location"),
                            unit=function_args.get("unit")
                        )

                        display(response)
                        tool_outputs.append({
                            "tool_call_id":toolcall.id,
                            "output":response
                        })


                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                            location=function_args.get("location")
                        )
                        tool_outputs.append({
                            "tool_call_id":toolcall.id,
                            "output":response
                        })
            print(tool_outputs)

            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
    elif runRequest.status == "completed":
        print("---------------------")
        print("succesfuly completed")
        print("---------------------")
        messages:list[ThreadMessage] =  client.beta.threads.messages.list(thread_id=thread.id)

        for message in reversed(messages.data):
            role_lable =  "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_lable}: {message_content} \n")
        break
    
    elif run.status == "failed":
        print(f"Run is {run.status}")
        break
    elif run.status in ["in_progress", "queued"]:
        print(f"Run is {run.status}. Waiting...")
        time.sleep(0.5)    

    else: 
        display(show_json(runRequest))
        break



Run is queued. Waiting...
requires_action ==========
Status:  requires_action


{'submit_tool_outputs': {'tool_calls': [{'id': 'call_CMMKkGVPybDigWM4S93UalCS',
    'function': {'arguments': '{"location":"Los Angeles, CA"}',
     'name': 'getCurrentWeather'},
    'type': 'function'}]},
 'type': 'submit_tool_outputs'}

toolCalls Present:
[RequiredActionFunctionToolCall(id='call_CMMKkGVPybDigWM4S93UalCS', function=Function(arguments='{"location":"Los Angeles, CA"}', name='getCurrentWeather'), type='function')]
getCurrentWeather


'{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'

[{'tool_call_id': 'call_CMMKkGVPybDigWM4S93UalCS', 'output': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'}]
Run is queued. Waiting...
---------------------
succesfuly completed
---------------------
User: how is the weather in Los Angles? 

Assistant: The current temperature in Los Angeles is 72°F. 

